**Merge and clean different interventions files**

In [ ]:
import pandas as pd
import geopandas as gpd
from owslib.wfs import WebFeatureService
import folium
import matplotlib.pyplot as plt
import mapclassify
from shapely.geometry import Point
import numpy as np

Loading datasets

In [ ]:
# ambulance          = pd.read_parquet('ambulance_locations.parquet.gzip', engine='pyarrow')
# mug                = pd.read_parquet('mug_locations.parquet.gzip', engine='pyarrow')
# pit                = pd.read_parquet('pit_locations.parquet.gzip', engine='pyarrow')
# aed                = pd.read_parquet('aed_locations.parquet.gzip', engine='pyarrow')
cad9               = pd.read_parquet('cad9.parquet.gzip', engine='pyarrow')
interventions_bxl  = pd.read_parquet('interventions_bxl.parquet.gzip', engine='pyarrow')
interventions_bxl2 = pd.read_parquet('interventions_bxl2.parquet.gzip', engine='pyarrow')
interventions1     = pd.read_parquet('interventions1.parquet.gzip', engine='pyarrow')
interventions2     = pd.read_parquet('interventions2.parquet.gzip', engine='pyarrow')
interventions3     = pd.read_parquet('interventions3.parquet.gzip', engine='pyarrow')

**Cleaning, appending interventions**

cad9

In [ ]:
# Drop columns that we decided not to use (poor quality/not present in other files etc.)
cad9.drop(columns=['province', 'Service Name', 'Permanence short name', 'Permanence long name', 'EventSubType Trip',
                   'EventLevel Trip', 'CitysectionName intervention', 'T1confirmed', 'Intervention time (T1Confirmed)', 
                   'Departure time (T1Confirmed)', 'UI', 'ID', 'MISSION_NR', 'AMBUCODE', 'UNIT_ID'], inplace=True)

In [ ]:
# Renaming remaining column names
cad9.rename(columns={'Mission ID': 'mission_id',
                   'Latitude permanence': 'latitude_permanence',
                   'Longitude permanence': 'longitude_permanence',
                   'Vector Type': 'vector_type',
                   'EventType Trip': 'eventtype_trip',
                   'CityName intervention': 'cityname_intervention',
                   'Latitude intervention': 'latitude_intervention',
                   'Longitude intervention': 'longitude_intervention',
                   'Province invervention': 'province_intervention',
                   'T0': 't0',
                   'T1': 't1',
                   'T2': 't2',
                   'T3': 't3',
                   'T4': 't4',
                   'T5': 't5',
                   'T6': 't6',
                   'T7': 't7',
                   'Name destination hospital' : 'name_destination_hospital',
                   'Intervention time (T1Reported)': 'intervention_time_t3t1',
                   'Departure time (T1Reported)': 'departure_time_t1t0'}, inplace=True)

In [ ]:
# Insert a column to track which file the data comes from
cad9.insert(0, 'source', 'cad9')

Brussels 1

In [ ]:
# Drop columns that we decided not to use (poor quality/not present in other files etc.)
interventions_bxl.drop(columns=['service_name', 'housenumber_permanence', 'permanence_short_name', 'permanence_long_name', 'eventLevel_firstcall',
                   'eventlevel_trip', 'postalcode_intervention', 't1confirmed', 't9', 'waiting_time', 'intervention_duration',
                   'unavailable_time', 'postalcode_destination_hospital', 'cityname_destination_hospital', 'streetname_destination_hospital', 
                   'housenumber_destination_hospital', 'calculated_traveltime_departure_', 'calculated_distance_departure_to',
                   'calculated_traveltime_destinatio', 'calculated_distance_destination_', 'number_of_transported_persons'], inplace=True)

In [ ]:
# Renaming remaining column names
interventions_bxl.rename(columns={'intervention_time_t1reported': 'intervention_time_t3t1',
                     'departure_time_t1reported': 'departure_time_t1t0'}, inplace=True)

In [ ]:
# Insert a column to track which file the data comes from
interventions_bxl.insert(0, 'source', 'interventions_bxl')

In [ ]:
# Add a column for province since this file doesn't have it
interventions_bxl.insert(1, 'province_intervention', 'BXL')

Brussels 2

In [ ]:
# Drop columns that we decided not to use (poor quality/not present in other files etc.)
interventions_bxl2.drop(columns=['description_nl', 'ic_description_nl', 'creationtime', 'Number of transported persons',
                   'Permanence long name NL', 'Permanence long name FR', 'Permanence short name NL', 'Permanence short name FR', 'Service Name NL',
                   'Service Name FR', 'Housenumber Permanence', 'Vector type FR', 'Cityname destination hospital', 
                   'Streetname destination hospital', 'Housenumber destination hospital', 'Abandon reason FR'], inplace=True)

In [ ]:
# Function to clean EventType and EventLevel - keep EventType only
def process_string(x):
    if x is None:
        return x
    elif '-' in x:
        first_dash_index = x.find('-')
        return x[:first_dash_index - 4] + x[first_dash_index:]
    else:
        return x

# Apply the function to the column
interventions_bxl2['EventType and EventLevel'] = interventions_bxl2['EventType and EventLevel'].apply(process_string)

In [ ]:
# Extract postal code and city name
interventions_bxl2['postalcode_permanence'] = interventions_bxl2['Cityname Permanence'].str.split().str[0]
interventions_bxl2['Cityname Permanence'] = interventions_bxl2['Cityname Permanence'].str.split().str[1:].str.join(' ')

In [ ]:
# Renaming remaining column names
interventions_bxl2.rename(columns={'Mission ID': 'mission_id',
                   'T0': 't0',
                   'Cityname Intervention': 'cityname_intervention',
                   'Longitude intervention': 'longitude_intervention',
                   'Latitude intervention': 'latitude_intervention',
                   'EventType and EventLevel': 'eventtype_firstcall',
                   'Cityname Permanence': 'cityname_permanence',
                   'Streetname Permanence': 'streetname_permanence',
                   'Latitude Permanence': 'longitude_permanence',
                   'Longitude Permanence': 'latitude_permanence',
                   'Vector type NL': 'vector_type',
                   'Name destination hospital': 'name_destination_hospital',
                   'Abandon reason NL': 'abandon_reason',
                   'T1': 't1',
                   'T2': 't2',
                   'T3': 't3',
                   'T4': 't4',
                   'T5': 't5',
                   'T6': 't6',
                   'T7': 't7'}, inplace=True)

In [ ]:
# Insert a column to track which file the data comes from
interventions_bxl2.insert(0, 'source', 'interventions_bxl2')

In [ ]:
# Add a column for province since this file doesn't have it
interventions_bxl2.insert(1, 'province_intervention', 'BXL')

Interventions 1

In [ ]:
# Drop columns that we decided not to use (poor quality/not present in other files etc.)
interventions1.drop(columns=['Service Name', 'HouseNumber permanence', 'Permanence short name', 'Permanence long name', 'EventLevel Firstcall',
                   'EventLevel Trip', 'PostalCode intervention', 'T1confirmed', 'T9', 'Intervention time (T1Confirmed)', 
                   'Departure time (T1Confirmed)', 'Unavailable time', 'PostalCode destination hospital', 'CityName destination hospital',
                   'StreetName destination hospital', 'HouseNumber destination hospital', 'Calculated travelTime destinatio',
                   'Calculated Distance destination', 'Number of transported persons', 'Waiting time', 'Intervention duration'], inplace=True)

In [ ]:
# Renaming remaining column names
interventions1.rename(columns={'Mission ID': 'mission_id',
                               'PostalCode permanence': 'postalcode_permanence',
                               'CityName permanence': 'cityname_permanence',
                               'StreetName permanence': 'streetname_permanence',
                               'Latitude permanence': 'latitude_permanence',
                               'Longitude permanence': 'longitude_permanence',
                               'Vector type': 'vector_type',
                               'EventType Firstcall': 'eventtype_firstcall',
                               'EventType Trip': 'eventtype_trip',
                               'CityName intervention': 'cityname_intervention',
                               'Latitude intervention': 'latitude_intervention',
                               'Longitude intervention': 'longitude_intervention',
                               'Province intervention': 'province_intervention',
                               'T0': 't0',
                               'T1': 't1',
                               'T2': 't2',
                               'T3': 't3',
                               'T4': 't4',
                               'T5': 't5',
                               'T6': 't6',
                               'T7': 't7',
                               'Intervention time (T1Reported)': 'intervention_time_t3t1',
                               'Departure time (T1Reported)': 'departure_time_t1t0',
                               'Name destination hospital': 'name_destination_hospital',
                               'Abandon reason': 'abandon_reason'}, inplace=True)

In [ ]:
# Insert a column to track which file the data comes from
interventions1.insert(0, 'source', 'interventions1')

Interventions 2

In [ ]:
# Drop columns that we decided not to use (poor quality/not present in other files etc.)
interventions2.drop(columns=['Service Name', 'HouseNumber permanence', 'Permanence short name', 'Permanence long name', 'EventLevel Firstcall',
                   'EventLevel Trip', 'PostalCode intervention', 'T1confirmed', 'T9', 'Intervention time (T1Confirmed)', 
                   'Departure time (T1Confirmed)', 'Unavailable time', 'PostalCode destination hospital', 'CityName destination hospital',
                   'StreetName destination hospital', 'HouseNumber destination hospital', 'Calculated travelTime destinatio',
                   'Calculated Distance destination', 'Number of transported persons', 'Waiting time', 'Intervention duration'], inplace=True)

In [ ]:
# Renaming remaining column names
interventions2.rename(columns={'Mission ID': 'mission_id',
                               'PostalCode permanence': 'postalcode_permanence',
                               'CityName permanence': 'cityname_permanence',
                               'StreetName permanence': 'streetname_permanence',
                               'Latitude permanence': 'latitude_permanence',
                               'Longitude permanence': 'longitude_permanence',
                               'Vector type': 'vector_type',
                               'EventType Firstcall': 'eventtype_firstcall',
                               'EventType Trip': 'eventtype_trip',
                               'CityName intervention': 'cityname_intervention',
                               'Latitude intervention': 'latitude_intervention',
                               'Longitude intervention': 'longitude_intervention',
                               'Province intervention': 'province_intervention',
                               'T0': 't0',
                               'T1': 't1',
                               'T2': 't2',
                               'T3': 't3',
                               'T4': 't4',
                               'T5': 't5',
                               'T6': 't6',
                               'T7': 't7',
                               'Intervention time (T1Reported)': 'intervention_time_t3t1',
                               'Departure time (T1Reported)': 'departure_time_t1t0',
                               'Name destination hospital': 'name_destination_hospital',
                               'Abandon reason': 'abandon_reason'}, inplace=True)

In [ ]:
# Insert a column to track which file the data comes from
interventions2.insert(0, 'source', 'interventions2')

Interventions 3

In [ ]:
# Drop columns that we decided not to use (poor quality/not present in other files etc.)
interventions3.drop(columns=['Service Name', 'HouseNumber permanence', 'Permanence short name', 'Permanence long name', 'EventLevel Firstcall',
                   'EventLevel Trip', 'PostalCode intervention', 'T1confirmed', 'T9', 'Intervention time (T1Confirmed)', 
                   'Departure time (T1Confirmed)', 'Unavailable time', 'PostalCode destination hospital', 'CityName destination hospital',
                   'StreetName destination hospital', 'HouseNumber destination hospital', 'Calculated travelTime destinatio',
                   'Calculated Distance destination', 'Number of transported persons', 'Waiting time', 'Intervention duration'], inplace=True)

In [ ]:
# Renaming remaining column names
interventions3.rename(columns={'Mission ID': 'mission_id',
                               'PostalCode permanence': 'postalcode_permanence',
                               'CityName permanence': 'cityname_permanence',
                               'StreetName permanence': 'streetname_permanence',
                               'Latitude permanence': 'latitude_permanence',
                               'Longitude permanence': 'longitude_permanence',
                               'Vector type': 'vector_type',
                               'EventType Firstcall': 'eventtype_firstcall',
                               'EventType Trip': 'eventtype_trip',
                               'CityName intervention': 'cityname_intervention',
                               'Latitude intervention': 'latitude_intervention',
                               'Longitude intervention': 'longitude_intervention',
                               'Province intervention': 'province_intervention',
                               'T0': 't0',
                               'T1': 't1',
                               'T2': 't2',
                               'T3': 't3',
                               'T4': 't4',
                               'T5': 't5',
                               'T6': 't6',
                               'T7': 't7',
                               'Intervention time (T1Reported)': 'intervention_time_t3t1',
                               'Departure time (T1Reported)': 'departure_time_t1t0',
                               'Name destination hospital': 'name_destination_hospital',
                               'Abandon reason': 'abandon_reason'}, inplace=True)

In [ ]:
# Insert a column to track which file the data comes from
interventions3.insert(0, 'source', 'interventions3')

Append interventions, clean further

In [ ]:
all_interventions = pd.concat([interventions1, interventions2, interventions3, interventions_bxl, interventions_bxl2, cad9], ignore_index=True)

In [ ]:
# Clean vector_type column
vector_type_mapping = {'Ambulance Event': 'Ambulance',
                       'Ambulance Disaster': 'Ambulance',
                       'MUG Event' : 'MUG',
                       'PIT Event' : 'PIT',
                       'Ambulance Exceptional' : 'Ambulance',
                       'MUG Disaster' : 'MUG',
                       'PIT Disaster' : 'PIT',
                       'Brandziekenwagen' : 'Ambulance',
                       'Decontanimatieziekenwagen' : 'Ambulance',
                       'AMB' : 'Ambulance'
                      }

all_interventions['vector_type'] = all_interventions['vector_type'].replace(vector_type_mapping)

In [ ]:
# Create a new column 'eventtype' which takes the value of 'eventtype_firstcall' when it is not missing, otherwise takes 'eventtype_trip'
all_interventions['eventtype'] = all_interventions['eventtype_firstcall'].fillna(all_interventions['eventtype_trip'])

# Drop the original columns
all_interventions = all_interventions.drop(columns=['eventtype_firstcall', 'eventtype_trip'])

In [ ]:
# Filter for cardiac eventtypes
cardiac_interventions = all_interventions[all_interventions['eventtype'].str.startswith(('P011', 'P003', 'P010',
                                                                                                 'P029', 'P004',
                                                                                                 'P019', 'P008',
                                                                                                 'P014')) | all_interventions['eventtype'].isna()]

In [ ]:
# Drop irrelevant time columns
cardiac_interventions.drop(columns=['t2', 't4', 't5', 't6', 't7'], inplace=True)

In [ ]:
# Clean intervention latitudes
# Step 1: Convert the column to string
cardiac_interventions['latitude_intervention'] = cardiac_interventions['latitude_intervention'].astype(str)

# Step 2: Remove decimal points from every value
cardiac_interventions['latitude_intervention'] = cardiac_interventions['latitude_intervention'].str.replace('.', '')

# Step 3: Add a point after every 2 characters
cardiac_interventions['latitude_intervention'] = cardiac_interventions['latitude_intervention'].apply(lambda x: x[:2] + '.' + x[2:])

# Step 4: Convert the column back to float64 data type
cardiac_interventions['latitude_intervention'] = pd.to_numeric(cardiac_interventions['latitude_intervention'], errors='coerce')

In [ ]:
# Clean intervention longitudes
# Step 1: Convert the column to string
cardiac_interventions['longitude_intervention'] = cardiac_interventions['longitude_intervention'].astype(str)

# Step 2: Remove decimal points from every value
cardiac_interventions['longitude_intervention'] = cardiac_interventions['longitude_intervention'].str.replace('.', '')

# Step 3: Add a point after every 2 characters
cardiac_interventions['longitude_intervention'] = cardiac_interventions['longitude_intervention'].apply(lambda x: x[:1] + '.' + x[1:])

# Step 4: Convert the column back to float64 data type
cardiac_interventions['longitude_intervention'] = pd.to_numeric(cardiac_interventions['longitude_intervention'], errors='coerce')

In [ ]:
# For some reason this is not working anymore....
# The format of t0 and t3 is different based on the source file --> we clean this, so that every format is the same
# def convert_datetime_t0(row):
#     if pd.isnull(row['t0']):
#         return None
#     elif row['source'] == 'cad9':
#         return pd.to_datetime(row['t0']).strftime('%Y-%m-%d %H:%M:%S')    # Convert to desired format for 'cad9'
#     elif row['source'] == 'interventions_bxl':
#         return pd.to_datetime(row['t0'], utc=True).strftime('%Y-%m-%d %H:%M:%S')    # Convert to desired format for 'bxl'
#     else:
#         return pd.to_datetime(row['t0'], format='%d%b%y:%H:%M:%S').strftime('%Y-%m-%d %H:%M:%S')    # Convert to default format for other values (interventions1, 2, 3, bxl2)

# def convert_datetime_t3(row):
#     if pd.isnull(row['t3']):
#         return None
#     elif row['source'] == 'interventions_bxl':
#         return pd.to_datetime(row['t3'], utc=True).strftime('%Y-%m-%d %H:%M:%S')    # Convert to desired format for 'bxl'
#     elif row['source'] == 'interventions_bxl2':
#         return pd.to_datetime(row['t3'], format='%d%b%y:%H:%M:%S').strftime('%Y-%m-%d %H:%M:%S')    # Convert to desired format for 'bxl2'
#     else:
#         return pd.to_datetime(row['t3']).strftime('%Y-%m-%d %H:%M:%S')    # Convert to default format for other values (interventions1, 2, 3, cad9)

# # Apply the functions to t0 and t2
# cardiac_interventions['t0'] = cardiac_interventions.apply(convert_datetime_t0, axis=1)
# # cardiac_interventions['t3'] = cardiac_interventions.apply(convert_datetime_t3, axis=1)


In [ ]:
# Ensure latitude and longitude are floats
cardiac_interventions['latitude_intervention'] = cardiac_interventions['latitude_intervention'].astype(float)
cardiac_interventions['longitude_intervention'] = cardiac_interventions['longitude_intervention'].astype(float)

# Convert to GeoDataFrame
gdf_cardiac = gpd.GeoDataFrame(
    cardiac_interventions,
    geometry=[Point(lon, lat) for lon, lat in zip(cardiac_interventions['longitude_intervention'], cardiac_interventions['latitude_intervention'])],
    crs="EPSG:4326"  # Assuming WGS84 (lat/lon)
)

Load Leuven and municipalities geodataframe

In [ ]:
# URL of the WFS service
wfs_url = "https://geo.api.vlaanderen.be/VRBG2019/wfs?service=WFS&version=2.0.0&request=GetFeature"

# Connect to the WFS service
wfs = WebFeatureService(wfs_url, version='1.1.0')

# List available layers
for layer in list(wfs.contents):
    print(layer)

# Choose the desired layer
layer_name = 'VRBG2019:Refgem'

# Fetch data as a GeoDataFrame
response = wfs.getfeature(typename=layer_name, outputFormat='json')

# Load the GeoDataFrame from the WFS response
flanders_municips_2019 = gpd.read_file(response)

In [ ]:
filtered_gdf = flanders_municips_2019[flanders_municips_2019['NISCODE'] == '24062']
filtered_gdf

In [ ]:
print("Filtered GDF CRS:", filtered_gdf.crs)
print("GDF Cardiac CRS:", gdf_cardiac.crs)

In [ ]:
gdf_cardiac = gdf_cardiac.to_crs(filtered_gdf.crs)

In [ ]:
# Perform spatial join to keep only points within the 'filtered_gdf' geometries
filtered_cardiac_interventions = gdf_cardiac[gdf_cardiac.geometry.within(filtered_gdf.union_all())]

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
filtered_gdf.plot(ax=ax, color='lightblue', edgecolor='black', alpha=0.5)
filtered_cardiac_interventions.plot(ax=ax, color='red', markersize=5, alpha=0.7)
plt.title("Intervention Points and Filtered Area")
plt.show()

Clean further

In [ ]:
filtered_cardiac_interventions = filtered_cardiac_interventions.drop(columns=['mission_id', 'postalcode_permanence', 'cityname_permanence',
                                                                              'streetname_permanence', 'latitude_permanence', 'longitude_permanence',
                                                                              'vector_type', 'name_destination_hospital', 'abandon_reason', 'geometry'])

In [ ]:
# Convert datetime columns to the desired format
def format_datetime(dt_value):
        if pd.isna(dt_value) or dt_value == '':
            return np.nan
        try:
            # Parse the datetime string and reformat it
            return pd.to_datetime(dt_value).strftime('%Y-%m-%d %H:%M:%S')
        except:
            # Return original value if conversion fails
            return dt_value

filtered_cardiac_interventions['t0'] = filtered_cardiac_interventions['t0'].apply(format_datetime)
filtered_cardiac_interventions['t3'] = filtered_cardiac_interventions['t3'].apply(format_datetime)

In [ ]:
# Again, this doesn't work...
# from datetime import datetime
# def format_custom_datetime(dt_value):
#         if pd.isna(dt_value) or dt_value == '':
#             return np.nan
#         # Parse using datetime.strptime with the specific format
#         # The format is: DDMMMYY:HH:MM:SS (01JUN22:10:27:09)
#         parsed_date = datetime.strptime(dt_value, '%d%b%y:%H:%M:%S')
        
#         # Convert to the desired format
#         return parsed_date.strftime('%Y-%m-%d %H:%M:%S')
    
# filtered_cardiac_interventions['t1'] = filtered_cardiac_interventions['t1'].apply(format_custom_datetime)

In [ ]:
# Calculate time between call and ambulance arrival (in minutes)
# Convert to datetime
filtered_cardiac_interventions['t3'] = pd.to_datetime(filtered_cardiac_interventions['t3'], errors='coerce')
filtered_cardiac_interventions['t0'] = pd.to_datetime(filtered_cardiac_interventions['t0'], errors='coerce')

# Function to calculate time difference (in seconds) based on alternative columns
def calculate_time_difference(row):
    if pd.isna(row['t3']) or pd.isna(row['t0']):
        # Use alternative columns if t3 or t0 is missing
        return (row['intervention_time_t3t1'] + row['departure_time_t1t0']) * 60
    else:
        # Calculate time difference based on t3 and t0
        return (row['t3'] - row['t0']).total_seconds()

# Apply the function to create the new column
filtered_cardiac_interventions['t3t0_diff'] = filtered_cardiac_interventions.apply(calculate_time_difference, axis=1)

In [ ]:
filtered_cardiac_interventions.to_excel("filtered_cardiac_interventions.xlsx", index=False)

In [ ]:
def remove_smart_duplicates(df):
    # Make a copy of the original dataframe 
    df_copy = df.copy()
    
    # Create a composite key from latitude and longitude
    df_copy['location_key'] = df_copy['latitude_intervention'].astype(str) + '_' + df_copy['longitude_intervention'].astype(str)
    
    # Create a score based on data completeness in t0 and t3
    # A row gets +1 for each non-null value in these columns
    df_copy['completeness_score'] = (
        df_copy['t0'].notna().astype(int) + 
        df_copy['t3'].notna().astype(int)
    )
    
    # Sort by completeness score in descending order
    # This ensures rows with more data appear first
    df_copy = df_copy.sort_values('completeness_score', ascending=False)
    
    # Keep the first occurrence of each location_key (which will be the most complete one)
    df_deduplicated = df_copy.drop_duplicates(subset=['location_key'])
    
    # Drop the temporary columns we created
    df_deduplicated = df_deduplicated.drop(columns=['location_key', 'completeness_score'])
    
    return df_deduplicated

In [ ]:
def remove_smart_duplicates_by_eventtype(df):
    # Create an empty dataframe to store the results
    result_df = pd.DataFrame(columns=df.columns)
    
    # Get unique event types
    event_types = df['eventtype'].unique()
    
    # Process each event type separately
    for event_type in event_types:
        # Get the subset of data for this event type
        event_df = df[df['eventtype'] == event_type].copy()
        
        # Create a composite key from latitude and longitude
        event_df['location_key'] = event_df['latitude_intervention'].astype(str) + '_' + event_df['longitude_intervention'].astype(str)
        
        # Create a score based on data completeness in t0 and t3
        # A row gets +1 for each non-null value in these columns
        event_df['completeness_score'] = (
            event_df['t0'].notna().astype(int) + 
            event_df['t3'].notna().astype(int)
        )
        
        # Sort by completeness score in descending order
        # This ensures rows with more data appear first
        event_df = event_df.sort_values('completeness_score', ascending=False)
        
        # Keep the first occurrence of each location_key within this event type
        event_df_deduplicated = event_df.drop_duplicates(subset=['location_key'])
        
        # Drop the temporary columns we created
        event_df_deduplicated = event_df_deduplicated.drop(columns=['location_key', 'completeness_score'])
        
        # Append to the result dataframe
        result_df = pd.concat([result_df, event_df_deduplicated], ignore_index=True)
    
    return result_df

In [ ]:
filtered_cardiac_interventions_clean = remove_smart_duplicates(filtered_cardiac_interventions)

In [ ]:
filtered_cardiac_interventions_clean.to_excel("interventions_new.xlsx", index=False)

In [ ]:
# cad9.to_excel("cad9.xlsx", index=False)